# Prophesee Event Camera 

In this tutorial we demonstrate how to obtain data from a Prophesee camera and stream the events into Lava. We show the basic usage of the PropheseeCamera Process and how to apply filters and transformations.

In [1]:
import matplotlib
%matplotlib inline

from lava.magma.core.run_configs import Loihi2SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.peripherals.dvs.prophesee import PropheseeCamera
from metavision_core.utils import get_sample
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm
from metavision_core.event_io import RawReader
from lava.lib.peripherals.dvs.transform import Compose, Downsample

from utils import VisProcess

/usr/lib/python3/dist-packages/metavision_core_ml/preprocessing/viz.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _viz_events(events, img):



## Download raw event recording

In [2]:
SEQUENCE_FILENAME_RAW = "80_balls.raw"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server
get_sample(SEQUENCE_FILENAME_RAW)

reader = RawReader(SEQUENCE_FILENAME_RAW)
height, width = reader.get_size()
del reader

## Basic usage

Initialize PropheseeCamera using the path to a recording file if RAW format, in order to use a live camera, replace the path with an empty string ("").


<table>
<tr>
    <td> <img src="gifs/basic.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [3]:

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

## Apply filters

As you see, the falling balls in the recording cause a trail of events. In many cases these trails are blurring the objects and are undesired. In order to reduce the trails, we can apply the TrailFilterAlgorithm from the metavision_sdk. See the metavision documentation for more filters and their usage.


<table>
<tr>
    <td> <img src="gifs/filters.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [4]:
filters = [TrailFilterAlgorithm(width=width, height=height, threshold=100000),
           ActivityNoiseFilterAlgorithm(width=width, height=height, threshold=1000),]


# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              filters=filters,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

## Apply transformations

We provide numerous transformations to be applied to the event data. For example Downsampling, MergePolarities and Mirroring.


<table>
<tr>
    <td> <img src="gifs/transform.gif" alt="Drawing" style="height: 125px;"/> </td>
</tr>
</table>

In [5]:
transformations = Compose(
    [
        Downsample(factor=0.5),
    ]
)

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

## Custom transformation and manual output shape

We also provide the option to use tonic transformations on the data. See the tonic documentation to see a complete list and usage. If you need to use custom transformation, the automatic shape determination would not work. In that case, you can specify the output shape manually.


<table>
<tr>
    <td> <img src="gifs/custom_trans.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [6]:
def expand_x_dim(events):
    events['x'] += 500
    return events
    

transformations = Compose(
    [
        expand_x_dim,
    ]
)

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1,
                              out_shape=(1, 2, height, width+500))

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()